# Test EV control

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ev_ssm import ev_ssm
from tqdm import tqdm
import scipy.stats as stats

In [2]:
# --- Response ---

sse = ev_ssm(ts=18, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA",
             n_pref=1, is_report=True,
             tt_mean=0.5, tt_var=0.2, tt_lb=0, tt_ub=1)
sse.load_A("Aest.csv")


/Users/jinningwang/Documents/work/andes/jwang/notes/ev_ssm.py:447: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ev['na'][self.ev['na'] > self.ev['nam']] = self.ev['nam'][self.ev['na'] > self.ev['nam']]
EVA: ts=18[H], 10000 EVs, Total Q=87.83 MWh
Online 2955, Q=73.98 MWh, SoC=0.7712
Power(MW): Pt=-8.1388, Pc=-8.1388, Pd=-0.0
Ctrl: 0=1581; 1=1374; 
EVA: Load A from Aest.csv.


In [ ]:
%run -i 'cosim_import.py'

In [ ]:
caseH = 18
%run -i 'cosim_setup_andes.py'

In [ ]:
%run -i 'cosim_loadsyn.py'

In [ ]:
%run -i 'cosim_setup_dispatch.py'

In [ ]:
%run -i 'cosim_linktable.py'

In [ ]:
# Change ``rru``, ``rrd``, ``rsfr``, ``t_total`` for necessary.
t_total = 3600

rru, rrd, rsfr = 1.0, 1.0, 0.04

# Define functions and constants used in loop
%run -i 'cosim_const.py'

Revise the EV control logic as: C->I, I->D, SOC high to low

In [ ]:
# --- Response ---

sse = ev_ssm(ts=18, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA")
sse.load_A("Aest.csv")


In [3]:
# --- Response ---

sse = ev_ssm(ts=10.4, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA")
sse.load_A("Aest.csv")

x0 = sse.x0.copy()
print(sse.Prc)
sse.run(tf=10.4 + 2/3600, Pi=9,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)
print(sse.Prc)
x1 = sse.x0.copy()

/Users/jinningwang/Documents/work/andes/jwang/notes/ev_ssm.py:447: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ev['na'][self.ev['na'] > self.ev['nam']] = self.ev['nam'][self.ev['na'] > self.ev['nam']]
EVA: ts=10.4[H], 10000 EVs, Total Q=44.91 MWh
Online 1697, Q=42.48 MWh, SoC=0.9737
Power(MW): Pt=-0.4919, Pc=-0.4919, Pd=-0.0
Ctrl: 0=1615; 1=82; 
EVA: Load A from Aest.csv.


0


EVA MCS: 100%|██████████| 3/3 [00:01<00:00,  2.54it/s]

9.37613161528854


In [ ]:
sse = ev_ssm(ts=10.4, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA",
             is_report=False)

# historical data
ev_num = pd.read_csv("ev_num.csv")

# calculate the sf
k1 = ev_num['ne'][(ev_num['time'] >= sse.ts) & (ev_num['time'] <= sse.ts+1/12)].mean()
k0 = ev_num['ne'][ev_num['time'] >= sse.ts].iloc[0]
k = k1 / k0

# estiamte FRC
sse.g_frc(nea=sse.ne*k)

In [ ]:
## Run to get the EV numbers within a day

# import joblib
# from joblib import Parallel, delayed

# Nt = 10000
# def rne(time):
#     sse1 = ev_ssm(ts=time, N=Nt, step=1, tp=40,
#                   lr=0.1, lp=60, seed=2022, name="EVA",
#                   is_report=False)
#     return sse1.ne

# # interval by minutes, 24H
# rnel = Parallel(n_jobs=joblib.cpu_count())(delayed(rne)(i) for i in np.arange(1/60, 24, 1/60))

# ev_num = pd.DataFrame()
# ev_num['time'] = np.arange(1/60, 24, 1/60)
# ev_num['ne'] = rnel
# ev_num.to_csv("ev_num.csv", index=False)